# Aufgabe 1 (Preliminaries)

This task requires a working installation of SQLite. The way to install it depends on your platform, but it is usually very easy. For Ubuntu 20.04 it’s just `sudo apt install sqlite3`

In [6]:
%load_ext sql

# Aufgabe 2 (University Schema)

Laden Sie die gegebene SQLite-Datenbank `university.db`

In [7]:
%sql sqlite:///university.db

1. Schauen Sie sich das Schema an:

        `.tables`

        `.schema --indent`

2. Schauen Sie sich die Instanz an (die Daten in jeder Tabelle):

    `select * from <table-name>;`

3. Vergleichen Sie mit dem in der Vorlesung dazu gegebenen Schema. Verstehen Sie die Bedeutung der Datensätze in allen diesen Tabellen, die Primary Keys und die Foreign-Key-Beziehungen zwischen den Tabellen. Beantworten Sie insbesondere folgende Fragen:

&emsp;&emsp;(a) Wie viele `instructors` kann ein `student` als `advisor` haben? Warum?

&emsp;&emsp;Entweder kein oder ein Instruktor. Die `advisor` Tabelle ist eine Zwischentabelle die als eine N zu M relation fungiert, was heisst jeder Student hat entweder genau einen oder keinen Instruktor.

&emsp;&emsp;(b) Wieviele `students` kann ein `instructor` als `advisor` haben? Warum?

```sql
CREATE TABLE advisor(
  s_ID  varchar(5),
  i_ID  varchar(5),
  primary key(s_ID),
  foreign key(i_ID) references instructor(ID)   on delete set null,
  foreign key(s_ID) references student(ID)      on delete cascade
);
```

&emsp;&emsp;Der `instructor` kann der `advisor` für beliebig viele `students` sein. Weil der Primary Key sich nur auf die `students.ID` bezieht und unique sein muss.<br>
&emsp;&emsp;Diesselbe `instructor.ID` kann mehrfach aufkommen in Datensätzen als `advisor.i_ID`


&emsp;&emsp;(c) Warum gibt es keine Foreign-Key-Beziehung von `section` auf `time_slot`?

&emsp;&emsp;Das Attribut `time_slot_id` ist in beiden Tabellen `section` und `time_slot` enthalten, jedoch ist es kein Primary Key in der `time_slot` Tabelle, demnach kann es auch keine Foreign-Key-Beziehung geben.

# Aufgabe 3 (Abfragen mit publications.db)

Es ist das umseitige Schema gegeben. Die zugehörige SQLite-Datenbank ist in `publications.db`. Geben Sie SQL-Queries für alle folgenden Aufgaben an. Testen Sie die Queries mit dieser Datenbank.

In [8]:
%sql sqlite:///publications.db

1. Finden Sie alle Autoren, die in San Francisco wohnen.

In [9]:
%%sql

select * from authors where city = 'San Francisco';

 * sqlite:///publications.db
   sqlite:///university.db
Done.


au_id,au_lname,au_fname,phone,address,city,state,country,postalcode
486-29-1786,Locksley,Chastity,415 585-4620,18 Broadway Av.,San Francisco,CA,USA,94130


2. Wieviele Titel beginnen mit 'S'?

In [10]:
%%sql

select count(*) as titles_starting_with_s from titles where title like 'S%';

 * sqlite:///publications.db
   sqlite:///university.db
Done.


titles_starting_with_s
4


3. Bestimmen Sie den durchschnittlichen Preis eines Titels.

In [11]:
%%sql

select avg(price) as average_price from titles;

 * sqlite:///publications.db
   sqlite:///university.db
Done.


average_price
14.76625


4. Geben Sie das Datum aller Verkäufe des Ladens 'Bookbeat' an.

In [12]:
%%sql

select sales.date from sales
  join stores on sales.stor_id = stores.stor_id
where stores.stor_name = 'Bookbeat';

 * sqlite:///publications.db
   sqlite:///university.db
Done.


date
1986-07-13 00:00:00
1989-05-23 00:00:00
1988-01-13 00:00:00
1991-03-20 00:00:00
1991-03-20 00:00:00
1991-03-20 00:00:00


5. Geben Sie alle Titel aus, die im Laden 'Bookbeat' verkauft wurden.

In [13]:
%%sql

select distinct titles.title from titles
  join salesdetail on titles.title_id = salesdetail.title_id
  join stores on salesdetail.stor_id = stores.stor_id
where stores.stor_name = 'Bookbeat';

 * sqlite:///publications.db
   sqlite:///university.db
Done.


title
Prolonged Data Deprivation: Four Case Studies
"Onions, Leeks, and Garlic: Cooking Secrets of the Mediterranean"
Life Without Fear
The Gourmet Microwave
Straight Talk About Computers
You Can Combat Computer Stress!
The Busy Executive's Database Guide
But Is It User Friendly?
Emotional Security: A New Algorithm
Fifty Years in Buckingham Palace Kitchens


6. Finden Sie Titel und Preis des teuersten Buches.

In [14]:
%%sql

select min(price) as cheapest, max(price) as most_expensive from titles;

 * sqlite:///publications.db
   sqlite:///university.db
Done.


cheapest,most_expensive
2.99,22.95


7. Finden Sie sämtliche Bücher, die teurer als das billigste Psychologie-Buch sind.

In [15]:
%%sql

select title from titles
where price > (
  select min(price) as cheapest from titles
  where type like 'psychology%'
);

 * sqlite:///publications.db
   sqlite:///university.db
Done.


title
Secrets of Silicon Valley
The Busy Executive's Database Guide
Emotional Security: A New Algorithm
Prolonged Data Deprivation: Four Case Studies
Cooking with Computers: Surreptitious Balance Sheets
Silicon Valley Gastronomic Treats
"Sushi, Anyone?"
Fifty Years in Buckingham Palace Kitchens
But Is It User Friendly?
Is Anger the Enemy?


8. Finden Sie die Autoren, die in einem Staat wohnen, in dem es keinen der erfassten Läden gibt.

In [16]:
%%sql

with states_without_stores(state) as (
    with authors_states(state) as (select distinct(a.state) from authors as a)
    select au_st.state from stores as stor
    RIGHT JOIN authors_states as au_st on stor.state = au_st.state
    WHERE stor.state IS NULL
)
SELECT a.au_fname as first_name, a.au_lname as last_name, a.state
FROM authors as a, states_without_stores
where a.state = states_without_stores.state;


 * sqlite:///publications.db
   sqlite:///university.db
Done.


first_name,last_name,state
Albert,Ringer,UT
Anne,Ringer,UT
Michel,DeFrance,IN
Sylvia,Panteley,MD
Morningstar,Greene,TN
Innes,del Castillo,MI
Meander,Smith,KS


9. Geben Sie die Städte an, in denen es sowohl Authoren wie auch Verleger gibt.

In [27]:
%%sql

WITH publisher_cities(city) as (
    SELECT distinct(city) from publishers
),
authors_cities(city) as (select distinct(a.city) from authors as a)
select authors_cities.city from authors_cities
join publisher_cities on authors_cities.city = publisher_cities.city;


 * sqlite:///publications.db
   sqlite:///university.db
Done.


city
Berkeley


10. Bestimmen Sie alle Bücher, die den gleichen Typ besitzen wie das Buch 'Net Etiquette'.

In [30]:
%%sql
Select title from titles
where type like (
    SELECT t.type from titles as t
    where t.title like 'Net Etiquette'
)

 * sqlite:///publications.db
   sqlite:///university.db
Done.


title
Secrets of Silicon Valley
But Is It User Friendly?
Net Etiquette


11. Geben Sie einen SQL Ausdruck an, der die Büchertypen zusammen mit der Anzahl Bücher jedes Typs ausgibt, von denen es mehr als 2 verschiedene Bücher gibt.

In [32]:
%%sql

SELECT t.type, count(t.type) as number_of_books
from titles as t
group by t.type
having number_of_books > 2

 * sqlite:///publications.db
   sqlite:///university.db
Done.


type,number_of_books
business,4
popular_comp,3
psychology,5
trad_cook,3


12. 

13. Geben Sie einen SQL Ausdruck an, der die Anzahl Autoren pro Staat auflistet, wobei die Ausgabe nach Anzahl Autoren sortiert sein soll.

In [37]:
%%sql

Select a.state, count(a.au_id) as number_of_authors
from authors as a
group by a.state
order by number_of_authors desc

 * sqlite:///publications.db
   sqlite:///university.db
Done.


state,number_of_authors
CA,15
UT,2
TN,1
OR,1
MI,1
MD,1
KS,1
IN,1


14. 